# BayesDB analysis of fish bouts in relation to paramecia characteristics

## 1. Ingesting the data

### a. Launching BayesDB

In [1]:
from csv_analysis import invert_all_bouts
import pandas as pd
#drct = '042318_6/'
drct = 'wik_bdb/'
csv_file = drct + 'all_huntbouts_marr.csv'
data = pd.read_csv(csv_file)
#invert_all_bouts(data, drct)


In [2]:
%load_ext jupyter_probcomp.magics

session_id: jovyan@nightcrawler2-notebook_2019-03-28T00:02:04.378307_9


In [3]:
%matplotlib inline
%vizgpm inline

<IPython.core.display.Javascript object>

In [4]:
!rm -f wik_bdb/bdb_hunts_marr.bdb
%bayesdb -j wik_bdb/bdb_hunts_marr.bdb



u'Loaded: wik_bdb/bdb_hunts_marr.bdb'

### b. Loading and printing the data from a `.csv` file

In [5]:
%sql DROP TABLE IF EXISTS bout_table;
#%bql CREATE TABLE bout_table FROM 'wik_bdb/huntbouts_inverted.csv' 
%bql CREATE TABLE bout_table FROM 'wik_bdb/all_huntbouts_marr.csv'

""


In [6]:
%sql SELECT * FROM bout_table LIMIT 5;

,Bout Number,Para Az,Para Alt,Para Dist,Postbout Para Az,Postbout Para Alt,Postbout Para Dist,Strike Or Abort
0,0.0,-0.977032,0.629180,354.291281,-0.175346,0.592776,299.960989,2.0
1,1.0,-0.162832,0.654781,305.528267,-0.158165,0.736650,286.473525,2.0
2,2.0,-0.179093,0.818221,281.527199,0.171628,0.271518,200.860390,2.0
3,3.0,0.073963,0.210280,134.098036,0.012218,0.210299,95.292186,2.0
4,4.0,0.108441,0.299536,77.905007,0.221962,0.746375,60.931606,2.0


In [7]:
%bql .nullify bout_table ''
%bql .nullify bout_table 'nan'

Nullified 0 cells
Nullified 156 cells


## 2. Automatically learning a CrossCat probabilistic model

### a. Defining an analysis population

In [8]:
%bql GUESS SCHEMA FOR bout_table

,column,stattype,num_distinct,reason
0,Bout Number,nominal,25.0,There are fewer than 20 distinct numerical va...
1,Para Az,numerical,1402.0,There are at least 20 unique numerical values...
2,Para Alt,numerical,1402.0,There are at least 20 unique numerical values...
3,Para Dist,numerical,1402.0,There are at least 20 unique numerical values...
4,Postbout Para Az,numerical,1349.0,There are at least 20 unique numerical values...
5,Postbout Para Alt,numerical,1349.0,There are at least 20 unique numerical values...
6,Postbout Para Dist,numerical,1349.0,There are at least 20 unique numerical values...
7,Strike Or Abort,nominal,2.0,There are fewer than 20 distinct numerical va...


In [9]:
%%mml
DROP POPULATION IF EXISTS bout_population;
CREATE POPULATION bout_population FOR bout_table WITH SCHEMA (GUESS STATTYPES OF (*);
                          IGNORE "Strike Or Abort", "Bout Number";)

""


### b. Creating and analyzing a probabilistic model (automatically)

In [10]:
%mml CREATE GENERATOR marr_generator FOR bout_population;

""


In [11]:
%multiprocess on

Multiprocessing turned on from on.


In [12]:
%mml INITIALIZE 50 MODELS IF NOT EXISTS FOR marr_generator;

""


In [13]:
%mml ALTER GENERATOR "marr_generator" ENSURE VARIABLES * DEPENDENT;

""


In [ ]:
%mml ANALYZE marr_generator FOR 100 ITERATIONS;
# note you can use the (OPTIMIZED) flag here but that gave you weird results last time. 

Completed: 100 iterations in 1248.275514 seconds.
Completed: 100 iterations in 1430.680435 seconds.
Completed: 100 iterations in 1338.850218 seconds.
Completed: 100 iterations in 1256.344098 seconds.
[================              ] 56.00%